<a href="https://colab.research.google.com/github/ndvp39/CloudComputing-tirgul/blob/main/index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests beautifulsoup4


In [ ]:
!pip install firebase
!pip install firebase-admin

In [ ]:
from google.colab import drive, output
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from collections import defaultdict
from firebase_admin import credentials
from firebase_admin import firestore
import firebase_admin
import requests
import re
import json
import os

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Function to extract text from a webpage
def get_page_text(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.get_text()
    except Exception as e:
        return ""


In [ ]:
# Function to extract links from a webpage
def get_links(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = [link.get('href') for link in soup.find_all('a', href=True)]
        return links
    except Exception as e:
        print("Error:", e)
        return []

In [ ]:
def remove_stop_words(text):
  stop_words = {'a','as','ha','thi','an', 'the', 'and', 'is', 'are', 'or', 'in', 'on', 'at','skip','-','them','they','than','she',',','.','&','for'}
  words = [word for word in text if word.lower() not in stop_words]
  return words

In [ ]:
def trim_words(words):
    cleaned_words = [word.rstrip(",.\\/") for word in words]
    return cleaned_words

In [ ]:
def apply_stemming(words):
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in words]
    return stemmed_words

In [ ]:
# Function to create index database
def create_index(url):
    index = defaultdict(list)
    visited = set()
    queue = [url]
    loop = 0
    while queue and loop < 40:
        loop+=1
        current_url = queue.pop(0)
        if current_url in visited:
            continue

        visited.add(current_url)
        text = get_page_text(current_url)
        words = remove_stop_words(text.split())
        words = trim_words(words)
        words = apply_stemming(words)

        for word in words:
            index[word].append(current_url)

        links = get_links(current_url)
        for link in links:
            if link.startswith(url) and link not in visited:
                queue.append(link)
    return index


In [ ]:
def update_dict(word, link):
    global link_ids
    if link not in link_ids:
        link_ids[link] = len(link_ids) + 1
    link_id = link_ids[link]
    if link not in index[word]:
        index[word][link] = {'id': link_id, 'counter': 0}
    index[word][link]['counter'] += 1

In [ ]:
def create_ranked_words(index):
    sorted_dict = {}
    counter_for_each_word = 0
    for word, links in index.items():
        for link in links.keys():
            counter_for_each_word += index[word][link]['counter']
        sorted_dict[word] = counter_for_each_word
        counter_for_each_word = 0
    sorted_dict = {k: v for k, v in sorted(sorted_dict.items(), key=lambda item: item[1], reverse=True)}
    ranked_dict = {}
    rank = 1
    for word, counter in sorted_dict.items():
        ranked_dict[word] = {'rank': rank, 'counter': counter}
        rank += 1
    return ranked_dict

In [ ]:
def create_data_for_db(final_index):
    data_list = []
    for word, doc_ids in final_index.items():
        term_data = {
            'term': word,
            'DocId': doc_ids
        }
        data_list.append(term_data)
    return data_list

In [ ]:
def generate_json_file(data_list, is_desktop):
    if is_desktop:
      filepath = os.path.join(os.path.dirname(os.path.abspath(__file__)), "indexDb.json")
    else:
      filepath = "/content/drive/My Drive/CloudComputing/project/indexDb.json"
    with open(filepath, "w") as json_file:
        json.dump(data_list, json_file, indent = 2)

In [ ]:
def print_collection_from_db(db, collection_name):
    collection_ref = db.collection(collection_name)
    docs = collection_ref.get()
    i = 0
    for doc in docs:
        term = doc.to_dict()['term']
        print(f"{i} -- Document ID: {doc.id}, Data: {term}")
        i += 1

In [ ]:
def upload_to_db(db, collection_name):
  try:
    for data in data_list:
        doc_ref = db.collection(collection_name).add(data)
    print("All documents uploaded successfully.")
  except Exception as e:
      print("Error:", e)

In [ ]:
website_url = 'https://www.redhat.com/en'
db_collection_name = "PantherIndex"

link_ids = {}
index_db = create_index(website_url)
index = defaultdict(dict)

for word, links in index_db.items():
    for link in links:
        update_dict(word, link)

ranked_words = create_ranked_words(index)
chosen_words = list(ranked_words.keys())[:110]
final_index = {word:index[word] for word in chosen_words}
data_list = create_data_for_db(final_index)

cred = credentials.Certificate("/content/drive/My Drive/CloudComputing/project/sak.json")
if not firebase_admin._apps:
  firebase_admin.initialize_app(cred)
db = firestore.client()

print_collection_from_db(db, db_collection_name)




Error: Exceeded 30 redirects.
Error: Exceeded 30 redirects.
Error: Exceeded 30 redirects.
Error: Exceeded 30 redirects.
Error: Exceeded 30 redirects.
Error: Exceeded 30 redirects.
Error: Exceeded 30 redirects.
Error: Exceeded 30 redirects.
0 -- Document ID: 0T4jk18lvS0dmor7kkEo, Data: triallearn
1 -- Document ID: 0dC3yFw12YLEqBoyIxaC, Data: buy
2 -- Document ID: 0rOjcv363Sdx7bxHCmXz, Data: from
3 -- Document ID: 14q8ET1SDE7sGorNMlJH, Data: organ
4 -- Document ID: 1OMqSz8Oy3XUhnh2eDe7, Data: sellr
5 -- Document ID: 239jIl6zY4B8DRyc8od2, Data: secur
6 -- Document ID: 2G9rtYGHlieDnuJOflVa, Data: across
7 -- Document ID: 2XtPvNMhEf42gc6jqrfb, Data: kubernet
8 -- Document ID: 37PzlFuvqdnSJ8DaMlQa, Data: how
9 -- Document ID: 3JlA8NiuJus3xFV9nlux, Data: technolog
10 -- Document ID: 3PUtUH4fOrAfqNvfzPjm, Data: build
11 -- Document ID: 49ptfniPoRUQURAU8Kug, Data: i
12 -- Document ID: 4kd7oscDguWFo1XAx1FM, Data: make
13 -- Document ID: 5iIeVk2PXs2CbEfJl1sH, Data: managementr
14 -- Document ID: 

Assingmnent 7 - ChatBot (**not for assingment 2**)

In [ ]:
def getLinkFromString(w,linkID):
  for word, links in final_index.items():
    if(word == w):
      return list(links.keys())[linkID]


!pip install nltk

import nltk
from nltk.chat.util import Chat, reflections

cloud_explanation = """Private Cloud: A private cloud is a cloud computing environment that is dedicated solely to one organization.
It is usually managed either by the organization itself or by a third-party service provider. The infrastructure and services are not shared
with other organizations, offering greater control, security, and customization options. Private clouds are typically used by large enterprises
or organizations with specific compliance or security requirements.
Public Cloud: A public cloud is a cloud computing environment that is hosted and managed by a third-party service provider and made available
to multiple organizations or individuals over the internet. The infrastructure and resources are shared among multiple users,
 offering scalability, flexibility, and cost-effectiveness. Public clouds are accessible to anyone who wants to use them and are often used
 for general-purpose computing needs such as web hosting, development, and testing.
Hybrid Cloud: A hybrid cloud is a computing environment that combines elements of both private and public clouds. It allows organizations
 to leverage the benefits of both environments by integrating on-premises infrastructure with public cloud resources. This integration enables
 data and applications to be dynamically moved between the two environments based on changing workload demands, security requirements, and cost
 considerations. Hybrid clouds offer greater flexibility, scalability, and customization options compared to using either private or public clouds alone.
 They are commonly used by organizations that want to retain control over sensitive data while taking advantage of the scalability and cost-effectiveness of
 public cloud services."""


# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Define some patterns and responses
patterns = [
    (r'hi|hello|hey', ['Hello!', 'Hi there!', 'Hey!']),
    (r'how are you?', ['I\'m good, thank you!', 'I\'m doing well, thanks for asking.']),
    (r'what is your name?', ['You can call me ChatGPT.', 'I go by the name ChatGPT.']),
    (r'your name (.*)', ['You can call me ChatGPT.', 'I go by the name ChatGPT.']),
    (r'give me good clud website', ['Sure, here: www.redhat.com', 'ofCourse: you can go to amazon, which works with RedHat website check both']),
    (r'What kinds of clouds are there ?', ['There are three main kinds of clouds: private, public, and hybrid would you like me to explain further? ', 'public, hybrid, private - in secured order']),
    (r'(.*)kinds of clouds(.*)?(.*)', ['There are three main kinds of clouds: private, public, and hybrid would you like me to explain further? ', 'public, hybrid, private - in secured order']),
    (r'please explain further about clouds(.*)', [cloud_explanation, "for the moment I too tired, may be later I will explain. You should've listen in class "]),
    (r'D|d(escribe) the presentor woman of R|r(ed)H|h(at)(.*)', ["she is taller than average american probably ", "She wears Pink"]),
    # You can add more patterns and responses here
]

for word, links in final_index.items():
  answers = []
  for i in range(len(links)):
    answers.append(getLinkFromString(word,i))
  to_append = ( r'(.*)' + word + r'(.*)', answers)
  patterns.append(to_append)

# Create a chatbot
chatbot = Chat(patterns, reflections)

# Start the conversation
print("Hello! I'm ChatGPT. How can I help you today?")
while True:
    user_input = input("You: ")
    response = chatbot.respond(user_input)
    print("ChatGPT:", response)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Hello! I'm ChatGPT. How can I help you today?
You: df fsfdfsdf dsfd hat dsfdfds
ChatGPT: she is taller than average american probably 
You: fgfg hat dfdf fd
ChatGPT: https://www.redhat.com/en/store
You: fgfg hat dfsdsd
ChatGPT: https://www.redhat.com/en/topics/devops/what-is-devsecops
